In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import openpyxl
from sklearn.linear_model import LinearRegression

# Load data

In [11]:
# import data
company_data_raw = pd.read_csv("data/GSFM_CRSP_Compustat_2006_-_2021.csv")
ceo_data_raw = pd.read_csv("data/py3mgn6xhyhchn42.csv")  # "data/Data_by_CEO.xlsx", )

/Users/jasmincapka/IdeaProjects/MADS/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (87,88,93) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [12]:
company_data_raw.head()

,GVKEY,LINKPRIM,LIID,LINKTYPE,LPERMNO,LPERMCO,LINKDT,LINKENDDT,datadate,fyear,...,priusa,sic,spcindcd,spcseccd,spcsrc,state,stko,weburl,dldte,ipodate
0,1004,P,1,LU,54594,20000,19720424,E,20070531,2006.0,...,1,5080.0,110.0,925.0,B,IL,0.0,www.aarcorp.com,NaN,19880101.0
1,1004,P,1,LU,54594,20000,19720424,E,20080531,2007.0,...,1,5080.0,110.0,925.0,B,IL,0.0,www.aarcorp.com,NaN,19880101.0
2,1004,P,1,LU,54594,20000,19720424,E,20090531,2008.0,...,1,5080.0,110.0,925.0,B,IL,0.0,www.aarcorp.com,NaN,19880101.0
3,1004,P,1,LU,54594,20000,19720424,E,20100531,2009.0,...,1,5080.0,110.0,925.0,B,IL,0.0,www.aarcorp.com,NaN,19880101.0
4,1004,P,1,LU,54594,20000,19720424,E,20110531,2010.0,...,1,5080.0,110.0,925.0,B,IL,0.0,www.aarcorp.com,NaN,19880101.0


In [13]:
company_data_raw.head()

,GVKEY,LINKPRIM,LIID,LINKTYPE,LPERMNO,LPERMCO,LINKDT,LINKENDDT,datadate,fyear,...,priusa,sic,spcindcd,spcseccd,spcsrc,state,stko,weburl,dldte,ipodate
0,1004,P,1,LU,54594,20000,19720424,E,20070531,2006.0,...,1,5080.0,110.0,925.0,B,IL,0.0,www.aarcorp.com,NaN,19880101.0
1,1004,P,1,LU,54594,20000,19720424,E,20080531,2007.0,...,1,5080.0,110.0,925.0,B,IL,0.0,www.aarcorp.com,NaN,19880101.0
2,1004,P,1,LU,54594,20000,19720424,E,20090531,2008.0,...,1,5080.0,110.0,925.0,B,IL,0.0,www.aarcorp.com,NaN,19880101.0
3,1004,P,1,LU,54594,20000,19720424,E,20100531,2009.0,...,1,5080.0,110.0,925.0,B,IL,0.0,www.aarcorp.com,NaN,19880101.0
4,1004,P,1,LU,54594,20000,19720424,E,20110531,2010.0,...,1,5080.0,110.0,925.0,B,IL,0.0,www.aarcorp.com,NaN,19880101.0


In [14]:
company_data_raw_columns = company_data_raw.columns
company_cols = ['GVKEY', 'prcc_f', 'ajex', 'ajp', 'fyear']

In [15]:
ceo_data_raw_columns = ceo_data_raw.columns
ceo_cols = ['GVKEY', 'CO_PER_ROL', 'YEAR', 'AGE', 'BECAMECEO', 'TITLE', 'PCEO']

In [16]:
# filter data
company_data = company_data_raw[company_cols]

ceo_data = ceo_data_raw[ceo_cols]
a = ceo_data[ceo_data.PCEO == "CEO"]

In [17]:
# join data
data_joined = ceo_data.join(company_data.set_index(['GVKEY', 'fyear']), on=['GVKEY', 'YEAR'], how='left', lsuffix='',
                            rsuffix='', sort=False)

# Preprocess data

In [18]:
# drop nan in prcc_f or ajex/ajp columns
data_joined.dropna(inplace=True)

data_joined.apply(lambda x: x.astype(str).str.lower())

,GVKEY,CO_PER_ROL,YEAR,AGE,BECAMECEO,TITLE,PCEO,prcc_f,ajex,ajp
45,1004,51547,2014,38.0,20180601.0,"ceo, president & director",ceo,29.54,1.0,1.0
51,1004,51547,2015,39.0,20180601.0,"ceo, president & director",ceo,24.41,1.0,1.0
57,1004,51547,2016,40.0,20180601.0,"ceo, president & director",ceo,34.94,1.0,1.0
62,1004,51547,2017,41.0,20180601.0,"ceo, president & director",ceo,44.69,1.0,1.0
66,1004,51547,2018,42.0,20180601.0,"ceo, president & director",ceo,30.09,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...
167905,326688,62002,2019,54.0,20180430.0,ceo & director,ceo,25.58,1.0,1.0
167912,326688,62002,2020,55.0,20180430.0,ceo & director,ceo,23.29,1.0,1.0
167921,328795,61546,2018,52.0,20181101.0,"president, ceo & director",ceo,27.69,1.0,1.0
167926,328795,61546,2019,53.0,20181101.0,"president, ceo & director",ceo,44.55,1.0,1.0


In [19]:
# Chairman, President, Founder

# founder - 1, otherwise 0
if_founder = data_joined['TITLE'].str.contains('founder')
data_joined['dummy_founder'] = if_founder

# president - 1, otherwise 0
if_president = data_joined['TITLE'].str.contains('president')
data_joined['dummy_president'] = if_founder

# chairman - 1, otherwise 0
if_chairman = data_joined['TITLE'].str.contains('chairman')
data_joined['dummy_chairman'] = if_founder

# chairman + president - 1, otherwise 0
data_joined['dummy_chairman_president'] = data_joined['TITLE'].str.contains('|'.join(['chairmam', 'president']))

In [20]:
# drop columns only important for joining
data_joined.drop(['GVKEY', 'CO_PER_ROL', 'PCEO', 'ajex', 'ajp', 'TITLE'], axis=1, inplace=True)

# Additional Features

In [21]:
# how many years as CEO - ceo_tenure
# how many years working there -
# percentage change in stock prices

# -> group for single CEOs - avg_change_in_sp

# Linear regression

In [22]:
X = data_joined.drop('prcc_f', axis=1)
y = data_joined['prcc_f']
lr = LinearRegression()

In [23]:
lr.fit(X, y)
lr.coef_

array([-8.57930153e+00,  1.24747785e+01, -1.61859352e-03, -7.34272025e+01,
       -7.34272025e+01, -7.34272025e+01, -2.64186270e+02])

In [24]:
coefficients = pd.concat([pd.DataFrame(X.columns), pd.DataFrame(np.transpose(lr.coef_))], axis=1)
coefficients

,0,0
0,YEAR,-8.579302
1,AGE,12.474778
2,BECAMECEO,-0.001619
3,dummy_founder,-73.427203
4,dummy_president,-73.427203
5,dummy_chairman,-73.427203
6,dummy_chairman_president,-264.186270
